In [ ]:
import chromadb
from chromadb.utils import embedding_functions

from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.pipeline.standard_pdf_pipeline import StandardPdfPipeline
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.datamodel.base_models import InputFormat

import asyncio
import logging
from aiogram import Bot, Dispatcher, types
from aiogram.filters import Command
from aiogram.methods import DeleteWebhook
from aiogram.types import Message

import nest_asyncio
nest_asyncio.apply()

import requests
import uuid
from datetime import datetime
import numpy as np
import re
import os

model = "deepseek/deepseek-v3-0324"

opts = PdfFormatOption(
    pipeline_cls=StandardPdfPipeline,
    backend=PyPdfiumDocumentBackend,
    pipeline_options=PdfPipelineOptions(do_ocr=True)
)
converter = DocumentConverter(format_options={InputFormat.PDF: opts})


In [2]:
# Открываем ПДФ файл, очищаем. 

def read_pdf_file(file_path):
    result = converter.convert(file_path, raises_on_error=False)
    document = result.document
    text = document.export_to_markdown()    
    text = re.sub(r"[\x00-\x1F\x7F-\x9F]+", "", text)            # Удалить квадраты и пустые знаки
    print(text)
    return text

In [3]:
# Делим текст на куски.

def text_to_chunk(text, chunk_size=250, overlap = 50):
    # Разделим текст с сохранением структуры предложения.
    sentences = text.replace('\n', ' ').split('. ')
    chunks = []
    current_chunk = []
    current_size = 0

    for sentence in sentences:
        # Ставим точку если срезали на середине.
        if not sentence.endswith('.'):
            sentence += '.'

        sentence_size = len(sentence)

        # Добавление предложения больше размера чанка?
        if current_size + sentence_size > chunk_size - overlap and current_chunk:
            chunks.append(' '.join(current_chunk))
            current_chunk = [sentence]
            current_size = sentence_size
        else:
            current_chunk.append(sentence)
            current_size += sentence_size

    # Добавляем элементы списка черех пробел
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    print("Чанк:", chunks)
    return chunks

In [4]:
# Запускаем работу БД.
client = chromadb.PersistentClient(path="./chroma_db")

sentence_transformer = embedding_functions.SentenceTransformerEmbeddingFunction(model_name = "all-MiniLM-L6-v2")
collection = client.get_or_create_collection(name = "condei_info", embedding_function = sentence_transformer)

In [5]:
# Главная функции. Обработка пдф документов из папки. Открытие, чтение, деление, внесение в БД.

def process_document(file_path):
    content = read_pdf_file(file_path)
    chunks = text_to_chunk(content)
    file_name = os.path.basename(file_path)
    metadatas = [{"Источник": file_name, "Чанк": i} for i in range(len(chunks))]
    ids = [f"{file_name}_чанк_{i}" for i in range(len(chunks))]
    return ids, chunks, metadatas

def add_to_collection(collection, ids, chunks, metadatas):
    batch_size = 100                                      # Вносим одновременно 100 токенов    
    for i in range(0, len(chunks), batch_size):
        end_idx = min(i + batch_size, len(chunks))
        collection.add(
            documents = chunks[i:end_idx],
            metadatas = metadatas[i:end_idx],
            ids = ids[i:end_idx]
        )

def process_and_add_documents(collection, folder_path):    
    files = [os.path.join(folder_path, file) 
             for file in os.listdir(folder_path) 
             if os.path.isfile(os.path.join(folder_path, file))]
    
    for file_path in files:
        print(f"Обрабатываю {os.path.basename(file_path)}...")        
        ids, chunks, metadatas = process_document(file_path)        
        add_to_collection(collection, ids, chunks, metadatas)        
        print(f"Добавлено {len(chunks)} чанков в БД")

In [ ]:
# Запуск главной функции обработки всех .pdf документов в папке /data.

process_and_add_documents(collection, "./data")

In [6]:
# Запуск семантического поиска и вывод чего нашли.

def semantic_search(collection, query, n_results):
    results = collection.query(
        query_texts = [query],
        n_results = n_results
    )

    # Просто нормально выводим итог семантического поиска
    print("\nРезультат поиска:\n" + "-" * 50)

    for i in range(len(results['documents'][0])):
        doc = results['documents'][0][i]
        meta = results['metadatas'][0][i]
        distance = results['distances'][0][i]
        print(f"\nРезультат {i + 1}: Источник: {meta['Источник']}, Чанк {meta['Чанк']}, Точность {round(distance, 3)}")
        print(f"Что нашли: {doc}\n")
    print("-" * 50)
    return results

In [7]:
# Фомируем информацию за запроса.

def get_context_for_query(results):
    context = "\n\n".join(results['documents'][0])
    sources = [
        f"{meta['Источник']} (Чанк {meta['Чанк']})"
        for meta in results['metadatas'][0]
    ]    
    return context, sources

In [8]:
# Подключаем ЛЛМ.

API_URL = "https://router.huggingface.co/novita/v3/openai/chat/completions"
headers = {"Authorization": f"Bearer {os.environ['HF_TOKEN']}"}

In [9]:
# ==========Добавляем возможность вести беседу=========

conversations = {}  # Заменяем на БД

def create_session():
    session_id = str(uuid.uuid4())
    conversations[session_id] = []
    return session_id

def add_message(session_id, role, content):
    if session_id not in conversations:
        conversations[session_id] = []

    conversations[session_id].append({
        "role": role,
        "content": content,
        "timestamp": datetime.now().isoformat()
    })

In [10]:
# Извлекаем историю общения и форматируем её в удобную для промпта строку.

def get_conversation_history(session_id, max_return_messages = None):
    if session_id not in conversations:
        return[]
    history = conversations[session_id]
    if max_return_messages:
        history = history[-max_return_messages:]
    return history

def format_history_for_include_in_prompt(session_id, max_return_messages = 5):
    history = get_conversation_history(session_id, max_return_messages)
    formatted_history = ""

    for msg in history:
        role = "user" if msg["role"] == "user" else "Assistant"
        formatted_history += f"{role}: {msg['content']}\n\n"
    print("Отформатированная история: ", formatted_history)
    return formatted_history

In [11]:
# Учим модель понимать смысл уточняющих вопросов и дополнять контекст

def contextualize_query(query, conversation_history):

    # Создаём отдельную базу запросов, основанную на истории и последнем запросе пользователя. Шаблон.    
    contextualize_q_system_prompt = (
        "Дано: история чата и последний вопрос пользователя, который может ссылаться на контекст из истории. Сформулируй самодостаточный (standalone) вопрос, не меняя названий, который будет понятен без истории чата. Не нужно отвечать на вопрос, только переформулируй его при необходимости. Если переформулировка не требуется — просто верни его как есть."
    )    
    payload = {
    "messages": [
        {   "role": "system", 
            "content": contextualize_q_system_prompt
        },
        {
            "role": "user",
            "content": f"История общения: {conversation_history} Вопрос: {query}"
        }
    ],
    "model": model,
    "temperature": 0.1    
    }  
    
    response = requests.post(API_URL, headers=headers, json=payload)    
    print("Ответ json:\n", response.json())
    inference = response.json()["choices"][0]["message"]    
    result = inference["content"]    
    return result    

In [12]:
# Получаем промпт с историей общения. 

def get_prompt(context, conversation_history, query):    
    prompt = f"""Используй контекст из документа и истории разговора чтобы дать развёрнутый ответ, не меняя названий. Если контекста из документа не релевантный, используй только историю разговора. Если контекст из документа не релевантный и история разговора отсутствует, напиши: Недостаточно информации. Пожалуйста, уточните вопрос.
    Контекст из документа:
    {context}
    История разговора:
    {conversation_history}

    Юзер: {query}

    Агент:"""    
    return prompt

In [13]:
# Обновляем функцию ответа историей общения

def generate_augmented_response(query, context, conversation_history = ""):    
    prompt = get_prompt(context, conversation_history, query)    
    payload = {
    "messages": [
        {   "role": "system", 
            "content": prompt},
        
    ],
    "model": model,
    "temperature": 0.1    
    }  
    response = requests.post(API_URL, headers=headers, json=payload)  
    inference = response.json()["choices"][0]["message"]
    print("Инфиренс: ", response.json())
    result = inference["content"]    
    return result 

In [14]:
# Объединяем финальный пайплайн для рага

def conversational_rag_query(
        collection, 
        query,
        session_id,
        n_chunks = 3
):
    # Получаем историю в нужном формате.
    conversation_history = format_history_for_include_in_prompt(session_id)    
    
    # Получаем историю в нужном формате.    
    query = contextualize_query(query, conversation_history)
    print("Запрос с контекстом: ", query)

    # Забираем нужные чанки.
    context, sources = get_context_for_query(
        semantic_search(collection, query, n_chunks)
    )      

    response = generate_augmented_response(query, context, conversation_history)

    # Добавляем в историю общения.
    add_message(session_id, "user", query)
    add_message(session_id, "assistant", response)

    return response, sources

In [ ]:
# Определяем сессию (новый пользователь или нет)

session_id = create_session()
print(session_id)

In [ ]:
# Делаем запрос руками, запускаем работу.
query = "Как включить продув испарителя на сплите dantex RK-24SVG"
response, sources = conversational_rag_query(collection, query, session_id)

print(response)

In [ ]:
# ======= Подключаем ТГ-бота ========
TOKEN='*'
logging.basicConfig(level=logging.INFO)
bot = Bot(TOKEN)
dp = Dispatcher()

# Обрабатываем команду старт
@dp.message(Command("start"))
async def cmd_start(message: types.Message):
    await message.answer("Здравия желаю! Бот-консультант по кондиционерам фирмы Daici&Dantex ждёт ваш запрос. Если ответ не приходит в течении 10 секунд, сервер отключен. Напишите автору — он починет.", parse_mode= "HTML")

# Обработчик текста
@dp.message(lambda message: message.text)
async def filter_messages(message: Message):
    query = message.text
    response, _ = conversational_rag_query(collection, query, session_id)
    await message.answer(response)


async def main():
    await bot(DeleteWebhook(drop_pending_updates=True))
    await dp.start_polling(bot)

try:
    loop = asyncio.get_running_loop()
except RuntimeError:
    loop = None

if loop and loop.is_running():
    if 'bot_task' in globals():
        print("⛔ Останавливаю старый polling...")
        bot_task.cancel()

    print("⚡ Event loop уже запущен, создаю задачу...")
    bot_task = loop.create_task(main())
else:
    # Обычный Python
    asyncio.run(main())


# if __name__ == "__main__":
#     try:
#         loop = asyncio.get_running_loop()
#     except RuntimeError:
#         loop = None

#     if loop and loop.is_running():
#         # уже есть активный event loop (например, Jupyter)
#         task = loop.create_task(main())
#     else:
#         asyncio.run(main()) 